In [2]:
import numpy as np
import librosa
from scipy.stats import entropy
import matplotlib.pyplot as plt

In [3]:
def calculate_entropy(elements):
    elements = np.asarray(elements)
    if elements.size == 0:
        return 0, 0
    value, counts = np.unique(elements, return_counts=True)
    probabilities = counts / counts.sum()
    ent = entropy(probabilities, base=2)
    max_entropy = np.log2(len(value)) if len(value) > 0 else 0
    relative_entropy = (ent / max_entropy * 100) if max_entropy > 0 else 0
    return ent, relative_entropy

def extract_features(audio_file, feature_type):
    y, sr = librosa.load(audio_file)
    if feature_type == 'pitch':
        pitches, _ = librosa.piptrack(y=y, sr=sr)
        pitches = pitches[pitches > 0]
        return np.round(librosa.hz_to_midi(pitches))
    elif feature_type == 'rhythm':
        onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
        onset_times = librosa.frames_to_time(onset_frames, sr=sr)
        return np.diff(onset_times) 
    elif feature_type == 'dynamics':
        S, _ = librosa.magphase(librosa.stft(y))
        rms = librosa.feature.rms(S=S)[0]
        rms = rms[rms > np.mean(rms) * 0.1]  
        rms = np.round(rms, 3) 
        return rms  
    elif feature_type == 'timbre':
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        spectral_centroids = spectral_centroids[spectral_centroids > np.mean(spectral_centroids) * 0.1] 
        spectral_centroids = np.round(spectral_centroids, -1)  
        return spectral_centroids 

def plot_histograms(features, feature_type, file_index, color):
    entropy_val, relative_entropy_val = calculate_entropy(features)
    fig, ax = plt.subplots(figsize=(10, 5))
    if feature_type == 'pitch':
        bins = 17 
    elif feature_type == 'timbre':
        bins = 25 
    else:
        bins = 30  
    ax.hist(features, bins=bins, color=color, alpha=0.75, edgecolor='black',
             label=f'{feature_type.capitalize()} - File {file_index + 1}: {entropy_val:.3f} bits, Relative: {relative_entropy_val:.2f}%')
    ax.set_title(f'{feature_type.capitalize()} Distribution for File {file_index + 1}')

    ax.set_xlim(left=0)

    if feature_type == 'pitch':
        ax.set_xlabel('Pitch value')
    elif feature_type == 'rhythm':
        ax.set_xlabel('Inter-onset Interval (seconds)')
    elif feature_type == 'dynamics':
        ax.set_xlabel('Root Mean Square Energy (amplitude)')
    elif feature_type == 'timbre':
        ax.set_xlabel('Spectral Centroid (Hz)')

    ax.set_ylabel('Frequency')
    ax.legend()

    fig.patch.set_linewidth(5)  
    fig.patch.set_edgecolor('black')  

    plt.show()

def main(audio_files, feature_type):
    colors = ['red', 'purple', 'orange', 'brown']
    for i, audio_file in enumerate(audio_files):
        features = extract_features(audio_file, feature_type)
        plot_histograms(features, feature_type, i, colors[i])




In [ ]:
audio_paths = ['audio1.wav']
metrics = ['pitch', 'rhythm', 'dynamics', 'timbre']
for metric in metrics:
    main(audio_paths, metric)